### 두개의 투자 전략을 하나의 class 선언

1. 클래스의 이름은 Invest 상속은 X
2. 생성자 함수 
    - 매개변수 4개
        - 주식 데이터 (_df)
        - 기준이 되는 컬럼의 이름 (_col = 'Adj Close')
        - 투자의 시작 시간 (_start = '2010-01-01')
        - 투자의 종료 시간 (_end = datetime.now())
    - 결측치, 양의 무한대, 음의 무한대를 제외
    - 컬럼중 Date 컬럼이 존재한다면 Date 컬럼을 인덱스로 변환
    - index를 시계열 데이터로 변환
    - index에 tz에 존재한다면 tz를 None 변경
    - 시작 시간과 종료 시간은 시계열 데이터로 변환
        - 종료시간은 문자열인 경우에만 변환환
    - 기준이 컬럼(self.col), 시작시간(self.start), 종료시간(self.end)은 self변수로 생성
3. buyandhold 함수를 생성 
    - 매개변수 x (self를 생성)
    - 시작시간과 종료시간으로 인덱스 필터 기준이되는 컬럼을 제외하고 모두 제거 
    - 일별 수익율 컬럼을 생성하여 pct_change() + 1 함수를 이용하여 값을 대입
    - 누적수익율 컬럼을 생성하여 누적 수익율 데이터를 대입 
    - 만들어진 데이터프레임과 총 누적수익율을 되돌려준다.
4. bollinger 함수를 생성 
    - 매개변수 2개 
        - 신뢰구간 (_num = 2)
        - 데이터 개수 (_cnt = 20)
    - 특정컬럼(self.col) 컬럼을 제외한 나머지 컬럼을 제거하고 변수에 저장
    - 이동평균선, 상단밴드, 하단밴드 생성
    - 시작시간, 종료 시간을 기준으로 데이터 필터링 
    - 보유 내역 추가 
    - 데이터 프레임을 되돌려준다. 
5. 수익율을 계산하는 함수 생성 create_rtn()
    - 매개변수 1개 : bollinger()함수의 결과 데이터프레임
    - 복사본 생성 
    - rtn 컬럼을 생성해서 1을 대입 
    - 매수, 매도 시기를 확인하여 수익율 대입 
    - rtn을 기준으로 누적 수익율을 acc_rtn 컬럼에 대입 
    - 만들어진 데이터프레임과 최종수익율을 되돌려준다.


In [1]:
import pandas as pd 
from datetime import datetime
import numpy as np

In [2]:
# momentum 모듈을 로드
import momentum as mmt

In [20]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
class Invest:
    # 생성자 함수 
    def __init__(
            self,
            _df, 
            _col = 'Adj Close', 
            _start = '2010-01-01', 
            _end = datetime.now()
    ):
        
        # 조건식 생성 
        flag = _df.isin( [np.nan, np.inf, -np.inf ] ).any(axis=1)
        self.df = _df.loc[~flag]
        # self.df의 컬럼중 Date가 존재한다면?
        if 'Date' in self.df.columns:
            self.df.set_index('Date', inplace=True)
        # index를 시계열 데이터로 변경 
        # index를 시계열로 변경할때 DatetimeIndex -> tz 가 존재 
        # index를 시계열로 변경할때 Index -> tz 존재 X
        # self.df.index = pd.DatetimeIndex(self.df.index, utc=True)
        self.df.index = pd.to_datetime(self.df.index)
        # tz가 존재하는가?
        try:
            self.df.index = self.df.index.tz_localize(None)
        except Exception as e:
            print(e)
        # 인자값으로 들어온 기준이 되는 컬럼의 이름은 self변수에 저장
        self.col = _col
        # 시작 시간은 시계열로 변경 
        try:
            self.start = datetime.strptime(_start, '%Y-%m-%d')
            if type(_end) == 'str':
                self.end = datetime.strptime(_end, '%Y-%m-%d')
            else:
                self.end = _end
        except:
            print("시간의 타입이 맞지 않습니다. 포멧은 YYYY-mm-dd 형식입니다.")
            print('Class를 다시 생성하시기 바랍니다.')
        
    def buyandhold(self):
        # 시작 시간, 종료시간 , 특정 컬럼을 기준으로 데이터를 필터 
        result = self.df.loc[self.start : self.end , [self.col]]
        # 일별 수익율 생성
        result['rtn'] = (result[self.col].pct_change() + 1).fillna(1)
        # 누적 수익율 생성 
        result['acc_rtn'] = result['rtn'].cumprod()
        # 총 누적 수익율을 변수에 저장 
        acc_rtn  = result.iloc[-1, -1]
        return result, acc_rtn
    
    def bollinger(
            self, 
            _num = 2, 
            _cnt = 20
    ):
        # 특정 컬럼을 기준으로 데이터를 필터
        result = self.df[[self.col]]
        # 이동 평균선 생성
        result['center'] = result[self.col].rolling(_cnt).mean()
        # 표준편차 값 변수에 저장 
        std_series = result[self.col].rolling(_cnt).std()
        # 상단밴드 생성
        result['up'] = result['center'] + (_num * std_series)
        # 하단밴드 생성
        result['down'] = result['center'] - (_num * std_series)
        # 시작 시간과 종료시간을 기준으로 데이터 필터
        result = result.loc[self.start : self.end]
        # 보유 내역 추가 
        result['trade'] = ''
        for idx in result.index:
            # 상단 밴드의 가격보다 기준이 되는 컬럼의 값이 크거나 같다면
            if result.loc[idx, self.col] >= result.loc[idx, 'up']:
                result.loc[idx, 'trade'] = ''
            # 하단밴드의 가격보다 기준이 되는 컬럼의 값이 작거나 같다면
            elif result.loc[idx, 'down'] >= result.loc[idx, self.col]:
                result.loc[idx, 'trade'] = 'buy'
            # 밴드 중간에 기준이되는 컬럼의 값이 존재한다면?
            else:
                # 현재 보유중이면 유지
                if result.shift().loc[idx, 'trade'] == 'buy':
                    result.loc[idx, 'trade'] = 'buy'
                # 보유중이 아니면 유지
                else:
                    result.loc[idx, 'trade'] = ''
                    
        # class 내부의 create_rtn 함수를 호출
        rtn_result, acc_rtn = self.create_rtn(result)
        return rtn_result, acc_rtn

    # 수익률을 계산하는 함수    
    def create_rtn(self, _df):
        # 복사본 생성
        result = _df.copy()
        
        # rtn 컬럼을 생성 1을 대입
        result['rtn'] = 1
        
        for i in result.index:
            # 매수
            if (result.shift().loc[i, 'trade'] == '') & (result.loc[i, 'trade'] == 'buy'):
                buy = result.loc[i, self.col]
                print(f"매수일 : {i}, 매수가 : {buy}")
                
            # 매도
            elif (result.shift().loc[i, 'trade'] == 'buy') & (result.loc[i, 'trade'] == ''):
                sell = result.loc[i, self.col]
                print(f"매도일 : {i}, 매도가 : {sell}")

                # 수익률 계산
                rtn = sell / buy
                # 수익률을 데이터프레임에 대입
                result.loc[i, 'rtn'] = rtn
                print(f"수익률 : {rtn}")
        # 누적수익률을 계산하고 대입    
        result['acc_rtn'] = result['rtn'].cumprod()
        # 총 누적수익률을 변수에 저장
        acc_rtn = result.iloc[-1, -1]
        return result, acc_rtn

    
    # momentum 함수를 생성
    # 인자 값 : 데이터(self.df), 기준이 되는 컬럼명(self.col), 시작시간(self.start),
    #           종료시간(self.end), _score = 1, _momentum = 12, _select = 1
    def momentum(
            self,
            _score = 1,
            _momentum = 12,
            _select = 1
    ) :
        # mmt모듈 안에 create_ym(데이터프레임, 기준이 되는 컬럼명) 함수를 호출 -> 변수에 저장
        ym_df = mmt.create_ym(self.df, _col = self.col)
        # mmt모듈 안에 create_month ( 데이터프레임, _select, _momentum, _start, _end) 함수를 호출
        month_df = mmt.create_month(ym_df,
                                    _start = self.start,
                                    _end = self.end,
                                    _select = _select,
                                    _momentum = _momentum)
        # 거래내역 함수를 호출하여 변수에 저장
        # create_trade(ym_df, month_df, _score)
        trade_df = mmt.create_trade(ym_df, month_df, _score = _score)
        # 수익률 계산
        # Invest class 안에 있는 create_rtn(데이터프레임)
        result, acc_rtn = self.create_rtn(trade_df)
        return result, acc_rtn
        

In [22]:
df = pd.read_csv('../../csv/AAPL.csv')

In [23]:
test_invest = Invest(df)

In [24]:
test_invest.buyandhold()

(             Adj Close       rtn   acc_rtn
 Date                                      
 2010-01-04   26.782711  1.000000  1.000000
 2010-01-05   26.829010  1.001729  1.001729
 2010-01-06   26.402260  0.984094  0.985795
 2010-01-07   26.353460  0.998152  0.983973
 2010-01-08   26.528664  1.006648  0.990515
 ...                ...       ...       ...
 2019-06-18  198.449997  1.023518  7.409631
 2019-06-19  197.869995  0.997077  7.387975
 2019-06-20  199.460007  1.008036  7.447342
 2019-06-21  198.779999  0.996591  7.421952
 2019-06-24  199.169998  1.001962  7.436514
 
 [2384 rows x 3 columns],
 7.436513727083075)

In [25]:
boll_df = test_invest.bollinger()

매수일 : 2010-01-22 00:00:00, 매수가 : 24.747818
매도일 : 2010-03-01 00:00:00, 매도가 : 26.154476
수익률 : 1.0568396777445188
매수일 : 2010-08-24 00:00:00, 매수가 : 30.026524
매도일 : 2010-09-08 00:00:00, 매도가 : 32.90366
수익률 : 1.0958198158401553
매수일 : 2011-03-16 00:00:00, 매수가 : 41.299767
매도일 : 2011-07-01 00:00:00, 매도가 : 42.957966
수익률 : 1.040150323366231
매수일 : 2011-11-14 00:00:00, 매수가 : 47.463268
매도일 : 2011-12-27 00:00:00, 매도가 : 50.876015
수익률 : 1.0719029081604747
매수일 : 2012-04-16 00:00:00, 매수가 : 72.601524
매도일 : 2012-06-18 00:00:00, 매도가 : 73.308609
수익률 : 1.0097392583659814
매수일 : 2012-10-08 00:00:00, 매수가 : 80.207954
매도일 : 2013-07-29 00:00:00, 매도가 : 57.243137
수익률 : 0.7136840443530077
매수일 : 2013-09-11 00:00:00, 매수가 : 60.184383
매도일 : 2013-10-18 00:00:00, 매도가 : 65.48336
수익률 : 1.0880457144505413
매수일 : 2014-01-03 00:00:00, 매수가 : 70.019096
매도일 : 2014-03-25 00:00:00, 매도가 : 70.960335
수익률 : 1.0134426042861222
매수일 : 2014-10-15 00:00:00, 매수가 : 89.842468
매도일 : 2014-10-23 00:00:00, 매도가 : 96.557182
수익률 : 1.0747387527243797
매수일 

In [26]:
momentum_df = test_invest.momentum()

날짜 : 2010-01-29 00:00:00, 모멘텀 인덱스 : 1.338067378873177, signal : buy
날짜 : 2010-02-26 00:00:00, 모멘텀 인덱스 : 1.1504870903585878, signal : buy
날짜 : 2010-03-31 00:00:00, 모멘텀 인덱스 : 0.9465371589200071, signal : buy
날짜 : 2010-04-30 00:00:00, 모멘텀 인덱스 : 0.8675996593182504, signal : buy
날짜 : 2010-05-28 00:00:00, 모멘텀 인덱스 : 0.9224656058655893, signal : buy
날짜 : 2010-06-30 00:00:00, 모멘텀 인덱스 : 0.8035523759459491, signal : buy
날짜 : 2010-07-30 00:00:00, 모멘텀 인덱스 : 0.5394459942740937, signal : buy
날짜 : 2010-08-31 00:00:00, 모멘텀 인덱스 : 0.5293378740562196, signal : buy
날짜 : 2010-09-30 00:00:00, 모멘텀 인덱스 : 0.3115730863758013, signal : buy
날짜 : 2010-10-29 00:00:00, 모멘텀 인덱스 : 0.5053051244562894, signal : buy
날짜 : 2010-11-30 00:00:00, 모멘텀 인덱스 : 0.505577053958894, signal : buy
날짜 : 2010-12-31 00:00:00, 모멘텀 인덱스 : 0.476534124094393, signal : buy
날짜 : 2011-01-31 00:00:00, 모멘텀 인덱스 : 0.6794752346651864, signal : buy
날짜 : 2011-02-28 00:00:00, 모멘텀 인덱스 : 0.658293625692578, signal : buy
날짜 : 2011-03-31 00:00:00, 모멘텀 인덱스 : 0.